In [46]:
import sys
from PIL import Image
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from gensim.corpora import Dictionary
from gensim.models import TfidfModel
from wordcloud import WordCloud
from collections import defaultdict

In [47]:
def format_sentence(sent):
    return({word: True for word in nltk.word_tokenize(sent)})

In [48]:
for x in pd.read_csv("review.csv",chunksize=10000):
    x.columns=['funny','user_id','review_id','text','business_id','stars','date','useful','cool']
    x.to_csv("out2.csv",index=False, mode='a')
    break

In [49]:
df = pd.read_csv("out2.csv")

In [50]:
import unicodedata

def strip_accents(text):
    try:
        text = unicode(text, 'utf-8')
    except NameError: # unicode is a default on python 3 
        pass
    text = unicodedata.normalize('NFD', text)
    text = text.encode('ascii', 'ignore')
    text = text.decode("utf-8")
    return str(text)

In [51]:
def get_common_surface_form(original_corpus, stemmer):
    counts = defaultdict(lambda : defaultdict(int))
    surface_forms = {}

    for document in original_corpus:
        for token in document:
            stemmed = stemmer.stem(token)
            counts[stemmed][token] += 1

    for stemmed, originals in counts.items():
        surface_forms[stemmed] = max(originals, 
                                     key=lambda i: originals[i])

    return surface_forms

stemmer = PorterStemmer() # Stemmer for reducing terms to root form 

stemmed_corpus = []       # For storing the stemmed tokens 

original_corpus = []      # For storing the non-stemmed tokens


for x in df['text']:    # Iterate over the files # Load file contents

    z= strip_accents(x)
    
    tokens = word_tokenize(z)     # Extract tokens

    stemmed = [stemmer.stem(token) for token in tokens] # Stem tokens

    stemmed_corpus.append(stemmed)    # Store stemmed document

    original_corpus.append(tokens)    # Store original document


dictionary = Dictionary(stemmed_corpus) # Build the dictionary


# Get the surface form for each stemmed word

counts = get_common_surface_form(original_corpus, stemmer)

# Convert to vector corpus

vectors = [dictionary.doc2bow(text) for text in stemmed_corpus]

# Train TF-IDF model

tfidf = TfidfModel(vectors)

# Get TF-IDF weights

weights = tfidf[vectors[0]]

# Replace term IDs with human consumable strings

weights = [(counts[dictionary[pair[0]]], pair[1]) for pair in weights]
weights=dict(weights)

# Initialize the cloud

wc = WordCloud(
    background_color="white",
    max_words=2000,
    width=1024,
    height=720,
    stopwords=stopwords.words('english')
)

# Generate the cloud

wc.generate_from_frequencies(weights)

# Save the cloud to a file

wc.to_file("wordcloud.png")

[(0, 0.047766189578841635), (1, 0.01099359730499089), (2, 0.012179222337392234), (3, 0.028689139829417426), (4, 0.005443014411188002), (5, 0.11351766625148363), (6, 0.11006329380189007), (7, 0.12704261082025087), (8, 0.1773411466579038), (9, 0.13469051928502018), (10, 0.06431551375482468), (11, 0.27243197952906734), (12, 0.021259446315209568), (13, 0.20089851284495985), (14, 0.12263277957123835), (15, 0.060145857649112), (16, 0.15499895963135182), (17, 0.1663489232017021), (18, 0.13852560326376437), (19, 0.12274704927658826), (20, 0.1106003965379117), (21, 0.023989560554101584), (22, 0.06223101407922806), (23, 0.3097703134308544), (24, 0.15045030124295977), (25, 0.12716729427731105), (26, 0.23752144638484665), (27, 0.1279219138361294), (28, 0.14510691228929867), (29, 0.29021382457859735), (30, 0.07921910373922296), (31, 0.136347794041056), (32, 0.1943738618436364), (33, 0.22347347952236676), (34, 0.04599569990291652), (35, 0.18046263620441272), (36, 0.17314716361561602), (37, 0.1075023